In [ ]:

# 06_evaluate_and_save_local.ipynb
%pip install -q numpy pandas scikit-learn joblib

import numpy as np, pandas as pd, joblib
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score
from pathlib import Path

PROC, EMB, MODELS = Path("data/processed"), Path("data/embeddings"), Path("models")

# LendingClub Hybrid
lc_test = pd.read_parquet(PROC/"lc_test.parquet")
NUM_COLS_LC = [c for c in lc_test.columns if c in ["annual_inc","dti","loan_amnt","zip_code","term","emp_length","revol_util","int_rate"]]
text_model = joblib.load(MODELS/"text_risk_model.pkl")
xgb_lc     = joblib.load(MODELS/"hybrid_model_lc.pkl")

X_emb = np.load(EMB/"lc_test_emb.npy")
text_score = text_model.predict_proba(X_emb)[:,1]
X_final = np.column_stack([lc_test[NUM_COLS_LC].values, text_score])
y = lc_test["default"].values

p = xgb_lc.predict_proba(X_final)[:,1]
auc = roc_auc_score(y, p); f1 = f1_score(y,(p>0.5).astype(int)); ap = average_precision_score(y,p)
print(f"LendingClub TEST: AUC={auc:.3f} | F1={f1:.3f} | PR-AUC={ap:.3f}")
print("Models saved locally in:", MODELS)
